### 7. Пример: форма для регистрации/авторизации

В данной секции мы рассмотрим реализацию формы для регистрации и авторизации пользователей в Django. Такие формы используются для создания новых пользователей (регистрация) или входа в систему (авторизация). Мы также затронем процесс валидации данных, таких как пароль и адрес электронной почты, и покажем пример использования `forms.ModelForm` для работы с моделью пользователя.

#### 7.1 Реализация формы для регистрации пользователя

Регистрация пользователя — это процесс создания нового пользователя в системе. Django предоставляет встроенную модель пользователя `User`, с которой можно работать через форму. Чтобы реализовать форму для регистрации, мы используем `forms.ModelForm`, что позволяет нам напрямую работать с моделью пользователя.

##### 7.1.1 Создание формы регистрации

Начнём с создания формы регистрации пользователя. Для этого нужно создать файл формы в вашем приложении. Например, создадим файл `forms.py`, если он ещё не существует.

**Шаг 1**: Создайте файл `forms.py` в вашем приложении и добавьте следующую реализацию формы:

```python
from django import forms
from django.contrib.auth.models import User
from django.contrib.auth.forms import UserCreationForm
from django.core.exceptions import ValidationError

class RegistrationForm(UserCreationForm):
    email = forms.EmailField(required=True, help_text="Введите корректный email.")
    
    class Meta:
        model = User
        fields = ['username', 'email', 'password1', 'password2']
    
    def clean_email(self):
        email = self.cleaned_data.get('email')
        if User.objects.filter(email=email).exists():
            raise ValidationError("Этот email уже зарегистрирован.")
        return email
```

##### Пояснения:
1. **`UserCreationForm`**: это встроенная форма Django, которая уже включает в себя поля для ввода имени пользователя и паролей.
2. **Переопределение поля `email`**: добавлено поле `email`, так как по умолчанию оно не является обязательным в модели пользователя, но для регистрации это поле часто важно.
3. **Переопределение метода `clean_email()`**: реализована кастомная валидация, которая проверяет, есть ли уже в базе данных пользователь с таким же email. Если пользователь с таким email уже существует, возвращается ошибка.


##### 7.1.2 Обработка формы в представлении

Теперь создадим представление для обработки формы регистрации. Представление будет обрабатывать запросы GET для отображения формы и POST для её отправки и проверки.

**Шаг 2**: Добавьте представление в файл `views.py` вашего приложения:

```python
from django.shortcuts import render, redirect
from .forms import RegistrationForm
from django.contrib import messages

def register(request):
    if request.method == 'POST':
        form = RegistrationForm(request.POST)
        if form.is_valid():
            form.save()
            messages.success(request, "Вы успешно зарегистрировались!")
            return redirect('login')
        else:
            messages.error(request, "Исправьте ошибки в форме.")
    else:
        form = RegistrationForm()
    
    return render(request, 'register.html', {'form': form})
```

##### Пояснения:
- **`form.is_valid()`**: проверяет, правильно ли заполнены все поля формы.
- **`form.save()`**: сохраняет нового пользователя, если форма прошла валидацию.
- **`messages.success` и `messages.error`**: используются для отображения сообщений о результатах регистрации.


##### 7.1.3 Шаблон для регистрации

Для отображения формы создадим шаблон **`register.html`**:

**Шаг 3**: Создайте файл `register.html` в директории шаблонов вашего приложения:

```html
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Регистрация</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container">
        <h2>Регистрация</h2>
        
        <form method="post">
            {% csrf_token %}
            {{ form.as_p }}
            <button type="submit" class="btn btn-primary">Зарегистрироваться</button>
        </form>
    </div>
</body>
</html>
```

Этот шаблон использует Bootstrap для базовой стилизации формы. Мы выводим все поля формы с помощью `form.as_p`, чтобы каждое поле было представлено в виде параграфа.


#### 7.2 Валидация пароля и электронной почты

Django предоставляет встроенные средства валидации для полей пароля и электронной почты, но иногда необходимо реализовать дополнительные проверки.

##### 7.2.1 Валидация электронной почты

Как упоминалось ранее, метод `clean_email()` в форме регистрации проверяет, зарегистрирован ли уже данный email. Это предотвращает использование одного и того же адреса электронной почты для создания нескольких аккаунтов.

```python
def clean_email(self):
    email = self.cleaned_data.get('email')
    if User.objects.filter(email=email).exists():
        raise ValidationError("Этот email уже зарегистрирован.")
    return email
```


##### 7.2.2 Валидация пароля

Форма `UserCreationForm` автоматически проверяет:
- Совпадение паролей в полях `password1` и `password2`.
- Сложность пароля: минимальная длина пароля и другие требования могут быть настроены через настройки проекта (`AUTH_PASSWORD_VALIDATORS`).

Вы можете настроить дополнительные правила проверки сложности пароля, используя встроенные валидаторы в `settings.py`:

```python
AUTH_PASSWORD_VALIDATORS = [
    {
        'NAME': 'django.contrib.auth.password_validation.UserAttributeSimilarityValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.MinimumLengthValidator',
        'OPTIONS': {
            'min_length': 8,
        }
    },
    {
        'NAME': 'django.contrib.auth.password_validation.CommonPasswordValidator',
    },
    {
        'NAME': 'django.contrib.auth.password_validation.NumericPasswordValidator',
    },
]
```

Эти валидаторы обеспечивают минимальную длину пароля, проверяют его сложность и уникальность.


#### 7.3 Пример работы с `forms.ModelForm`

В предыдущем разделе мы использовали `ModelForm` для создания формы регистрации пользователя. Вот основные шаги:

1. **Использование модели**: форма основывается на модели `User`.
2. **Поля формы**: автоматически создаются поля для всех полей модели, но вы можете настроить, какие именно поля будут отображены, с помощью атрибута `fields`.
3. **Кастомизация**: если нужно добавить кастомные поля (например, email) или валидацию, вы можете это сделать, переопределяя методы формы.

##### Пример создания кастомной формы с дополнительными полями:

```python
class CustomUserForm(forms.ModelForm):
    password = forms.CharField(widget=forms.PasswordInput)
    
    class Meta:
        model = User
        fields = ['username', 'email', 'password']
    
    def clean_password(self):
        password = self.cleaned_data.get('password')
        if len(password) < 8:
            raise forms.ValidationError("Пароль должен быть не менее 8 символов.")
        return password
```

Здесь мы добавили поле пароля и кастомную валидацию, проверяющую его длину.
